# Use TF-IDF to provide weighted keywords for movie plot

Need add more comments

In [3]:
# Load libraries
import sys 
print(sys.executable) 
import pandas as pd
import nltk 
nltk.download('stopwords')


C:\Users\Serena Jin\Anaconda3\python.exe


[nltk_data] Downloading package stopwords to C:\Users\Serena
[nltk_data]     Jin\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
# Load movie plot data
df_idf = pd.read_csv("movie.tsv",sep='\t')

df_idf['plot'].astype(str)
df_idf['title'].astype(str)

df_idf.head()


,title,plot
0,Dumb and Dumber To,20 years after the dimwits set out on their fi...
1,The Giant Mechanical Man,An offbeat romantic comedy about a silver-pain...
2,Outpost: Black Sun,A pair of investigators team up with a Special...
3,My Sister's Keeper,Sara and Brian live an idyllic life with their...
4,"I, Frankenstein","200 years after his shocking creation, Dr. Fra..."


In [13]:
import re
# from string import ascii_lowercase

def pre_process(text):
    
#     text=text.lower()
        
    return text

df_idf['text'] = df_idf['title'] + df_idf['plot']
# df_idf['text'] = df_idf['plot']

df_idf['text'] = df_idf['text'].apply(lambda x:pre_process(x))
 
#show the second 'text'
# df_idf['text'][10]
df_idf['text'][:10]

0    Dumb and Dumber To20 years after the dimwits s...
1    The Giant Mechanical ManAn offbeat romantic co...
2    Outpost: Black SunA pair of investigators team...
3    My Sister's KeeperSara and Brian live an idyll...
4    I, Frankenstein200 years after his shocking cr...
5    Lobos de ArgaAfter 15 years, Tomas, a not very...
6    ももへの手紙Clinging to an unfinished letter written...
7    Harold's Going StiffLonely pensioner, Harold G...
8    Eddie: The Sleepwalking CannibalA once-famous ...
9    Ain't in It for My Health: A Film About Levon ...
Name: text, dtype: object

In [14]:
from sklearn.feature_extraction.text import CountVectorizer
import re
import numpy as np
from stop_words import get_stop_words

#load a set of stop words, 
# stopwords=get_stop_words("stopwords.txt")
stopwords = get_stop_words('en')
 
cv=CountVectorizer(max_df=0.85,stop_words=stopwords, max_features=1000)

word_count_vector = cv.fit_transform(df_idf['plot'].apply(lambda x: np.str_(x)))

In [23]:
# docs=df_idf['text'].tolist()
docs=word_count_vector
docs

<16756x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 241064 stored elements in Compressed Sparse Row format>

In [26]:
 
list(cv.vocabulary_.keys())[:5]

['20', 'years', 'set', 'first', 'adventure']

In [25]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(word_count_vector)

TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)

In [21]:
docs_test=df_idf['text'].tolist()
docs_test

['Dumb and Dumber To20 years after the dimwits set out on their first adventure, they head out in search of one of their long lost children in the hope of gaining a new kidney.',
 'The Giant Mechanical ManAn offbeat romantic comedy about a silver-painted street performer and the soft spoken zoo worker who falls for him.',
 'Outpost: Black SunA pair of investigators team up with a Special Forces Unit to venture deep inside a war raging between the military and a massive army of Nazi Zombie Stormtroopers. Their mission is to fight their way behind enemy lines, locate the technology at the source of this growing threat and prevent the seemingly inevitable rise of the 4th Reich.',
 "My Sister's KeeperSara and Brian live an idyllic life with their young son and daughter. But their family is rocked by sudden, heartbreaking news that forces them to make a difficult and unorthodox choice in order to save their baby girl's life. The parents' desperate decision raises both ethical and moral ques

In [23]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)
 
def extract_topn_from_vector(feature_names, sorted_items, topn=5):
    """get the feature names and tf-idf score of top n items"""
    
    #use only topn items from vector
    sorted_items = sorted_items[:topn]
 
    score_vals = []
    feature_vals = []
    
    # word index and corresponding tf-idf score
    for idx, score in sorted_items:
        
        #keep track of feature name and its corresponding score
        score_vals.append(round(score, 3))
        feature_vals.append(feature_names[idx])
 
    #create a tuples of feature,score
    #results = zip(feature_vals,score_vals)
    results= {}
    for idx in range(len(feature_vals)):
        results[feature_vals[idx]]=score_vals[idx]
    
    return results

In [51]:
# you only needs to do this once, this is a mapping of index to 
feature_names=cv.get_feature_names()
 
# get the document that we want to extract keywords from
doc=docs_test[5]

 
#generate tf-idf for the given document
tf_idf_vector=tfidf_transformer.transform(cv.transform([doc]))
 
#sort the tf-idf vectors by descending order of scores
sorted_items=sort_coo(tf_idf_vector.tocoo())
 
#extract only the top n; n here is 10
keywords=extract_topn_from_vector(feature_names,sorted_items,20)
 
# now print the results
print("\n=====Doc=====")
print(doc)
print("\n===Weighted Keywords===")
for k in keywords:
    print(k,keywords[k])


=====Doc=====
Lobos de ArgaAfter 15 years, Tomas, a not very succesfull writer, comes back to the village in Galicia where his family comes from, Arga, apparently to get an award. But he doesn't know the real reason why he is there, to end with an old curse that is been haunting the village for the last hundred years.

===Weighted Keywords===
village 0.337
tomas 0.313
comes 0.31
hundred 0.245
apparently 0.242
years 0.239
curse 0.232
haunting 0.229
reason 0.22
15 0.209
award 0.207
de 0.204
writer 0.188
doesn 0.178
know 0.172
real 0.159
end 0.159
last 0.155
get 0.135
back 0.133
